In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import numpy as np
import torchvision
from torchvision import datasets, models, transforms

import time
import os
from PIL import Image
from tempfile import TemporaryDirectory

from torch.utils.data import DataLoader, SubsetRandomSampler
import random
from torch.utils.data import random_split

!wandb login 2a9ca7406b5b4c61d59f0f3606a221268edc9ca8
import wandb


In [ ]:
save_model_path = '/kaggle/working/ResNet50_customed_4_augument.pt'

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class CustomDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.classes = self._find_classes()
        self.image_paths, self.labels = self._load_data()

    def _find_classes(self):
        classes = sorted([d for d in os.listdir(self.data_dir) if os.path.isdir(os.path.join(self.data_dir, d))])
        return classes

    def _load_data(self):
        image_paths = []
        labels = []
        for label in self.classes:
            class_dir = os.path.join(self.data_dir, label)
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                image_paths.append(img_path)
                labels.append(int(label))
        return image_paths, labels

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        label = int (label)
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
train_data_dir = '/kaggle/input/augument-dataset/wb_recognition_dataset/train/images'
val_data_dir = '/kaggle/input/augument-dataset/wb_recognition_dataset/val/images'

normalize = transforms.Normalize(mean=[0.56,0.56,0.535], std=[0.02,0.02,0.02])

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
#     transforms.RandomRotation(degrees=15),
#     transforms.ColorJitter(),
    transforms.ToTensor(),
    normalize
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    normalize
])

train_dataset = CustomDataset(data_dir=train_data_dir, transform=train_transform)
val_dataset = CustomDataset(data_dir=val_data_dir, transform=val_transform)

image_datasets = {"train":train_dataset, "val": val_dataset}

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers = 4, pin_memory = True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False,num_workers = 4, pin_memory = True)

dataloader = {"train": train_loader, "val": val_loader}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
print (dataset_sizes['train'])
print (dataset_sizes['val'])
print (len(image_datasets['val'].classes))
# for images, labels in dataloader["train"]:
#     images, labels = images.to(device), labels.to(device)
#     # Now `images` and `labels` are moved to CUDA if available

# # Similarly for validation data loader
# for images, labels in dataloader["val"]:
#     images, labels = images.to(device), labels.to(device)

In [ ]:
model = torchvision.models.resnet50(weights='IMAGENET1K_V2')
# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2139)

for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# # Decay LR by a factor of 0.1 every 7 epochs
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:

def train_model(model, criterion, optimizer, scheduler, num_epochs, save_model_path, dataloader):

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloader[phase]:
                
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
#             if phase == 'train':
#                 scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            phase_acc, phase_loss = phase + "acc", phase + "loss"
            wandb.log({str(phase_acc): epoch_acc, str(phase_loss):epoch_loss})
        torch.save(model.state_dict(), save_model_path)
    return model

In [ ]:

wandb.init(project='ImageProcessing-project', sync_tensorboard=True)
try:
    model.load_state_dict(torch.load(save_model_path))
except:
    f = open(save_model_path, 'w')
    print("initial file")

model = train_model(model, criterion, optimizer, scheduler,
                       300, save_model_path, dataloader)
wandb.finish()